# Weather compilation project

University of Dayton - Environmental Protection Agency Average Daily Temperature Archive,
http://academic.udayton.edu/kissock/http/Weather/default.htm 

above link hosts average Daily temperature for many countries, Will start to explore and persome some analysis on the same.

<h2>Brief description of weather data and sources</h2>

<p><br>
This archive contains files of average daily temperatures for 157 <st1:country-region
w:st="on"><st1:place w:st="on">U.S.</st1:place></st1:country-region> and 167 international
cities. Source data for these files are from the Global Summary of the Day
(GSOD) database archived by the National Climatic Data Center (NCDC).<span
style='mso-spacerun:yes'>&nbsp; </span>The average daily temperatures posted on
this site are computed from 24 hourly temperature readings in the Global
Summary of the Day (GSOD) data.</p>

<p>The data fields in each file posted on this site are: month, day, year,
average daily temperature (F).<span style='mso-spacerun:yes'>&nbsp; </span>We
use &quot;-99&quot; as a no-data flag when data are not available. </p>

<div class=MsoNormal align=center style='text-align:center'>

<hr size=2 width="100%" align=center>

</div>

<p class=MsoNormal><o:p>&nbsp;</o:p></p>

<h2>Comparison of &#8220;24 hour&#8221; and &#8220;(min + max) / 2&#8221;
average daily temperatures</h2>

<p class=MsoNormal><o:p>&nbsp;</o:p></p>

<p class=MsoNormal>The average daily temperatures posted on this site are from
Global Summary of the Day (GSOD) dataset and are computed from 24 hourly
temperature readings. The GSOD dataset also includes daily minimum and maximum
temperatures.<span style='mso-spacerun:yes'>&nbsp; </span>Some earlier datasets
compiled by the NCDC, such as the Local <span class=SpellE>Climatological</span>
Data Monthly Summary, contained daily minimum and maximum temperatures, but did
not contain the average daily temperature computed from 24 hourly
readings.<span style='mso-spacerun:yes'>&nbsp; </span>As a result, some users
calculated the average daily temperature as the average of the daily minimum and
maximum temperatures.</p>

<p>We compared average daily temperatures calculated from 24 hourly readings,
T24, to average daily temperatures calculated as the average of the daily
minimum and maximum temperatures, <span class=SpellE>Tminmax</span>, for 53,004
daily temperature records in the GSOD dataset.<span
style='mso-spacerun:yes'>&nbsp; </span>We found that, on average, the absolute
value of the deviation between T24 and <span class=SpellE>Tminmax</span> was
1.48 F.<span style='mso-spacerun:yes'>&nbsp; </span>In addition, we found that,
on average, <span class=SpellE>Tminmax</span> was 0.0790 F higher than T24.
Temperatures in the GSOD dataset are reported with a precision of 0.1 F.<span
style='mso-spacerun:yes'>&nbsp; </span>Thus, the average bias is less than the
precision of the source data, and we conclude that the bias between T24 and <span
class=SpellE>Tminmax</span> is not statistically significant.<span
style='mso-spacerun:yes'>&nbsp; </span>If the bias is negligible, then the
deviation is random and will sum to zero over any sufficiently long time
period.<span style='mso-spacerun:yes'>&nbsp; </span>Thus, use of either T24 or <span
class=SpellE>Tminmax</span> &#8220;average&#8221; daily temperatures should
give similar results.<span style='mso-spacerun:yes'>&nbsp;&nbsp;&nbsp; </span></p>

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Plan at this point of time
1. Analyse the page understand the page setup
2. Then use bs4 to get the link download the contents and extract get a dataframe out of this.
3. Then lets see how it progresses

In [2]:
# collecting Non US data first
url = 'https://academic.udayton.edu/kissock/http/Weather/citylistWorld.htm'
response = requests.get(url)

soup = BeautifulSoup(response.content, features="lxml")


In [3]:
cities = soup.find_all("li",class_="MsoNormal")
# base_url = "https://academic.udayton.edu/kissock/http/Weather/"  # commenting as the way data pull is changed.
all_cities = []
# all_links = []
all_filename = []
all_countries = []
for city in cities:
    city_name = city.text.split("(")[0].rstrip()
    all_cities.append(city_name)
    # link = city.find('a')['href']    # commenting as the way data pull is changed.
    filename = city.find('a')['href'].split('/')[-1]
    all_filename.append(filename)
    # page is having multiple a links in each case but the city is presendin in just 1 a tag so did some stupid workaround which works!!
    country = city.find_all("a")[-1].text[:2].strip() + city.find_all("a")[0].text[:2].strip() 
    # all_links.append(base_url + link)   # commenting as the way data pull is changed.
    all_countries.append(country[:2])
# print(len(all_cities))
# print(len(all_filename))
# print(len(all_countries))


### change of plan


Got to know a single place to download the data but there are some conuntry code and US City code conflict. so preparing them 1ft
- So going with only non US Cities first < br >
- One that is completed then will work on US cities


In [4]:
# collecting US data
url = 'https://academic.udayton.edu/kissock/http/Weather/citylistUS.htm'
response = requests.get(url)
soup = BeautifulSoup(response.content, features="lxml")
cities = soup.find_all("li", class_="MsoNormal")
all_UScities = []
all_USfilename = []
all_USstate = []
for city in cities:
    city_name = city.text.split("(")[0].rstrip()
    all_UScities.append(city_name)
    # link = city.find('a')['href']    # commenting as the way data pull is changed.
    filename = city.find('a')['href'].split('/')[-1]
    all_USfilename.append(filename)
    # page is having multiple a links in each case but the city is presendin in just 1 a tag so did some stupid workaround which works!!
    state = city.find_all("a")[-1].text[:2].strip() + \
        city.find_all("a")[0].text[:2].strip()
    # all_links.append(base_url + link)   # commenting as the way data pull is changed.
    all_USstate.append(state[:2])
print(len(all_UScities))
print(len(all_USfilename))
print(len(all_USstate))

158
158
158


### Site is slow, So sending 300+ requests(Mulitple time when testing does not seems to fair) will be huge so planned to download the entire data use the file name to read from file specifically.

So steps to do now..

1. Download the entire file with all the data
2. Unzip the file to a data directory
3. fileformat 
   1. US Data --> SSCCCCCC.txt (SS-State abbreviation CCCCCC-City abbreviation up to 6 letters). 
   2. Internatinal data -->  SSCCCCCC.txt (SS-Country abbreviation CCCCCC-City abbreviation up to 6 letters).
4. create two dataframe one for US and other international with datetime series as index, colums will each of the city 
5. Get country code - counntry reff and US state code and States name from a relaible source for plotting
6. Then begin with plotting to see if there are any pattern
7. other steps will be determined futher

In [5]:
# importing the requests module
import requests
url = "https://academic.udayton.edu/kissock/http/Weather/gsod95-current/allsites.zip"

if os.path.exists('allsites.zip'):
    print("file Already present")
else:
    print('Downloading started')
    # Downloading the file by sending the request to the URL
    req = requests.get(url)

    # Split URL to get the file name
    filename = url.split('/')[-1]

    # Writing the file to the local file system
    with open(filename, 'wb') as output_file:
        output_file.write(req.content)
    print('Downloading Completed')


file Already present


In [6]:
import zipfile
filename = "allsites.zip"
print(filename)
try:
    with zipfile.ZipFile(filename) as z:
        z.extractall('data/03Temperature')
        print("Extracted all")
except:
    print("Invalid file")


allsites.zip
Extracted all


In [7]:
# County code https://www.iban.com/country-codes
import pandas as pd
url = "https://www.iban.com/country-codes"

country_code = pd.read_html(url)[0]

country_code


,Country,Alpha-2 code,Alpha-3 code,Numeric
0,Afghanistan,AF,AFG,4
1,Åland Islands,AX,ALA,248
2,Albania,AL,ALB,8
3,Algeria,DZ,DZA,12
4,American Samoa,AS,ASM,16
...,...,...,...,...
244,Wallis and Futuna,WF,WLF,876
245,Western Sahara,EH,ESH,732
246,Yemen,YE,YEM,887
247,Zambia,ZM,ZMB,894


In [17]:
url = r'https://knowledgecenter.zuora.com/BB_Introducing_Z_Business/D_Country%2C_State%2C_and_Province_Codes/B_State_Names_and_2-Digit_Codes'

US_State_code = pd.read_html(url)[0]
US_State_code


,State or Region Code,Name
0,AA,Armed Forces America
1,AE,Armed Forces
2,AK,Alaska
3,AL,Alabama
4,AP,Armed Forces Pacific
5,AR,Arkansas
6,AZ,Arizona
7,CA,California
8,CO,Colorado
9,CT,Connecticut


## cleanup of the read txt file

Below function createcsv will read the txt files extracted and coverets to a CSV, with correct date foramt and the temperature!

In [8]:
import re
import datetime
import os
import pandas as pd

def createcsv(path):
    """Received one parameter, convers the these files to a proper CSV! """
    templist = []
    file = open(path, "r")
    if path[-3:] == "LOG":
        return 0
    print(f"started {path}", end=" **")
    while(True):
        # read next line
        line = file.readline()
        # check if line is not null
        if not line or line.strip() == "":
            break
        numbers = re.findall('[-.0-9]+', line)
        if int(numbers[1]) == 0:  # Dirty xix to address a erorr date issue in CUHAVANA.txt
            # Dirty xix to address a erorr date issue in CUHAVANA.txt and DLHAMBUR.txt
            numbers[1] = 29
        if int(numbers[2]) == 201:
            continue
        # print(numbers)
        col1 = datetime.date(
            int(numbers[2]), int(numbers[0]), int(numbers[1]))
        col2 = float(numbers[-1])
        # print(col1, col2)
        templist.append([col1,col2])
    # print(templist[:5])
    print(end=" ** ")
    tempdf = pd.DataFrame(templist,columns=['date', 'temp'])
    # print(path[:-3]+"csv")
    # tempdf.to_csv("00_deleteme.csv",index=False)
    tempdf.to_csv(path[:-3]+"csv", index=False)
    print(end="**")
    print(end="**")
    file.close()
    print(end=" ** ")
    os.remove(path)
    print(" done with " + path[19:-3]+"csv")


# createcsv('data/03Temperature/ABTIRANA.txt')


In [9]:
# Importing the os library
import os
# The path for listing items
path = 'data/03Temperature'
# The list of items
files = os.listdir(path)
# Loop to print each filename separately
for filename in files:
    full_path = path+'/'+filename
    createcsv(full_path)


started data/03Temperature/ABTIRANA.txt ** ** **** **  done with ABTIRANA.csv
started data/03Temperature/AGBUENOS.txt ** ** **** **  done with AGBUENOS.csv
started data/03Temperature/AKANCHOR.txt ** ** **** **  done with AKANCHOR.csv
started data/03Temperature/AKFAIRBA.txt ** ** **** **  done with AKFAIRBA.csv
started data/03Temperature/AKJUNEAU.txt ** ** **** **  done with AKJUNEAU.csv
started data/03Temperature/ALALGIER.txt ** ** **** **  done with ALALGIER.csv
started data/03Temperature/ALBIRMIN.txt ** ** **** **  done with ALBIRMIN.csv
started data/03Temperature/ALHUNTSV.txt ** ** **** **  done with ALHUNTSV.csv
started data/03Temperature/ALMOBILE.txt ** ** **** **  done with ALMOBILE.csv
started data/03Temperature/ALMONTGO.txt ** ** **** **  done with ALMONTGO.csv
started data/03Temperature/ARFTSMIT.txt ** ** **** **  done with ARFTSMIT.csv
started data/03Temperature/ARLIROCK.txt ** ** **** **  done with ARLIROCK.csv
started data/03Temperature/AUBRSBAN.txt ** ** **** **  done with

# Below this point -- Work still going on!!


In [ ]:
get 

In [10]:
import pandas as pd
path = 'data/03Temperature'
# The list of items
files = os.listdir(path)
# Loop to print each filename separately
all_csvs = [path+'/'+filename for filename in files]
print(len(all_csvs))
no1stCSV = all_csvs.pop()

####### WORK ON the picking only non us files and then pick correct column name and perform left join.
df_nonUS = pd.read_csv(no1stCSV, index_col="date")
print(df_nonUS.head())


print(len(all_csvs))

# left = pd.DataFrame({
#     'Sr': [6, 7, 8, 9, 2],
#     'Name': ['Span', 'Suchu', 'Vetts', 'Appu', 'Sri']})
# right = pd.DataFrame({
#     'Sr': [6, 7, 8, 9, 20],
#     'Name': ['fil', 'mil', 'sil', 'pil', 'gil']})
# print(pd.merge(left, right, on='Sr', how='outer'))


326
            temp
date            
1995-01-01  73.4
1995-01-02  70.4
1995-01-03  72.7
1995-01-04  76.3
1995-01-05  76.8
325


In [11]:
import pandas as pd
left = pd.DataFrame({
    'Sr': [6, 7, 8, 9, 2],
    'Name': ['Span', 'Suchu', 'Vetts', 'Appu', 'Sri']}) #, index='Sr')
left.set_index('Sr')
right = pd.DataFrame({
    'Sr': [6, 7, 8, 9, 20],
    'Name': ['fil', 'mil', 'sil', 'pil', 'gil']}) #,index='Sr')
right.set_index('Sr')
# print(pd.merge(left, right, on='Sr', how='outer'))


,Name
Sr,
6,fil
7,mil
8,sil
9,pil
20,gil


In [12]:
pd.merge(left, right, left_index=True, right_index=True)
left.join()


TypeError: join() missing 1 required positional argument: 'other'